# Panoptic Quality : supplementary materials

This notebook supplements the paper:

A. Foucart, O. Debeir, C. Decaestecker. "Why Panoptic Quality should be avoided as a metric for assessing cell nuclei segmentation and classification in digital pathology", 2022.

## Downloading and pre-processing the data

To reproduce the results from this notebook, you need to:

* Download the multi-rater evaluation dataset [CSV files]([Drive with CSV files](https://drive.google.com/drive/folders/16P04eKeX3n5oRx3MxODISXaRTbHU22U0))) from the [NuCLS challenge](https://sites.google.com/view/nucls/multi-rater?authuser=0) and put them in the "./nucls_csv" directory.
* Download the MoNuSAC [test set annotations & teams predictions](https://monusac-2020.grand-challenge.org/Data/) and set them in the "./monusac_annotations" and "./monusac_teams" directories.
* Generate "nary masks" from the MoNuSAC annotations & teams predictions for easier processing afterwards using the code below. This operation may take a while. Requires [Openslide](https://openslide.org/). 

In [ ]:
from monusac import generate_nary_masks_from_annotations, generate_nary_masks_from_teams

generate_nary_masks_from_annotations("./monusac_annotations")
generate_nary_masks_from_teams("./monusac_teams")

## 1. NuCLS

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from nucls import load_annotations, match_experts

NUCLS_PATH = "./nucls_csv"

annotations, all_slides = load_annotations(NUCLS_PATH)
matches = match_experts(annotations, all_slides)

In [ ]:
bins = [500, 1000, 1500, 2000, 5000]
binned = [[] for _ in bins]

for match in matches:
    for i,b in enumerate(bins):
        if match.area < b:
            binned[i].append(match.iou)
            break
            
plt.figure(figsize=(10,4))
plt.boxplot(binned)
plt.xticks(range(1, 6), [f'<{b}' for b in bins])
plt.xlabel('Area (px)')
plt.ylabel('IoU')
plt.show()

In [ ]:
from skimage.measure import find_contours
from skimage.io import imread
import os
from metrics import compute_iou, compute_hd

path_single = "./example_single_cell"

rgb = imread(os.path.join(path_single, 'rgb.png'))
gt = imread(os.path.join(path_single, 'gt.png'))>0
seg_1 = imread(os.path.join(path_single, 'seg_1.png'))>0
seg_2 = imread(os.path.join(path_single, 'seg_2.png'))>0
seg_3 = imread(os.path.join(path_single, 'seg_3.png'))>0
seg_4 = imread(os.path.join(path_single, 'seg_4.png'))>0

ref_contours = find_contours(gt)[0]

plt.figure(figsize=(15,5))
for i,mask in enumerate([seg_1, seg_2, seg_3, seg_4]):
    contours = find_contours(mask)[0]
    plt.subplot(1, 4, i+1)
    plt.imshow(rgb)
    plt.plot(ref_contours[:, 1], ref_contours[:, 0], 'b-')
    plt.plot(contours[:, 1], contours[:, 0], 'k--')
    plt.title(f'IoU={compute_iou(gt, mask):.2f}')
plt.show()

In [ ]:
plt.figure(figsize=(15,7))
for match in matches:
    plt.plot(match.iou, match.hd, 'b+')
plt.plot([0, 1], [3, 3], 'k-')
plt.text(0, 4, 'HD=3')
plt.xlabel('IoU')
plt.ylabel('HD')
plt.show()

## 2. MoNuSAC experiments

In [ ]:
from monusac import get_all_nuclei

MONUSAC_ANNOTATIONS_PATH = "./monusac_annotations/"
MONUSAC_TEAMS_PATH = "./monusac_teams/"

nuclei = get_all_nuclei(MONUSAC_ANNOTATIONS_PATH)

Experiment on all nuclei: eroded, dilated and shifted

In [ ]:
from skimage.morphology import erosion, dilation, disk
from tqdm import tqdm
import numpy as np

se = disk(1)

ious = {}

for cl_type, cl_nuclei in nuclei.items():
    ious[cl_type] = []
    for nucleus in tqdm(cl_nuclei):
        eroded = erosion(nucleus.mask, se)
        dilated = dilation(nucleus.mask, se)
        shifted = np.zeros_like(nucleus.mask)
        shifted[1:] = nucleus.mask[:-1]
        ious[cl_type].append((compute_iou(nucleus.mask, eroded),
                              compute_iou(nucleus.mask, dilated),
                              compute_iou(nucleus.mask, shifted),
                              nucleus.area))

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

plt.figure(figsize=(10,5))
plt.boxplot([[iou[3] for iou in ious[cl]] for cl in ious])
plt.xticks([1, 2, 3, 4], [cl for cl in ious])
plt.ylabel('Area (px)')
plt.show()

In [ ]:
print("Median and interquartile range:")
for cl in ious:
    areas = [iou[3] for iou in ious[cl]]
    areas.sort()
    q25 = int(0.25*len(areas))
    q75 = int(0.75*len(areas))
    print(f"{cl}: {np.median(areas)} [{areas[q25]} - {areas[q75]}]")
    

In [ ]:
plt.figure(figsize=(20,5))
plt.boxplot([[iou[0] for iou in ious[cl]] for cl in ious], positions=[1, 5, 9, 13], showfliers=False)
plt.boxplot([[iou[1] for iou in ious[cl]] for cl in ious], positions=[2, 6, 10, 14], showfliers=False)
plt.boxplot([[iou[2] for iou in ious[cl]] for cl in ious], positions=[3, 7, 11, 15], showfliers=False)
plt.xticks([1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 14, 15], ['Ee', 'Ed', 'Es', 'Le', 'Ld', 'Ls', 'Ne', 'Nd', 'Ns', 'Me', 'Md', 'Ms'])
plt.ylabel('IoU')
plt.show()

In [ ]:
print("Median and interquartile range:")
for cl in ious:
    e = [iou[0] for iou in ious[cl]]
    d = [iou[1] for iou in ious[cl]]
    s = [iou[2] for iou in ious[cl]]
    e.sort()
    d.sort()
    s.sort()
    
    q25 = int(0.25*len(e))
    q75 = int(0.75*len(e))
    
    print(cl)
    print(f"Eroded: {np.median(e):.2f} [{e[q25]:.2f} - {e[q75]:.2f}]")
    print(f"Dilated: {np.median(d):.2f} [{d[q25]:.2f} - {d[q75]:.2f}]")
    print(f"Shifted: {np.median(s):.2f} [{s[q25]:.2f} - {s[q75]:.2f}]")

Checking the results of the different team per nucleus:

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
from monusac import show_nucleus_team_comparison

# Checking results for a single nucleus:
to_show = [
    ['Epithelial', 2],
    ['Lymphocyte', 15],
    ['Neutrophil', 5],
    ['Macrophage', 28]
]
# Epithelial - 2 -> IoU < 0.5 for very good segmentation
# Lymphocyte - 15 -> bad IoU for good segmentation + better IoU for worse shape
# Neutrophil - 5 -> nearly identical segmentation, large range of scores
# Macrophage - 28 -> class mismatch means worse PQ than missed detection

for cl, idn in to_show:
    show_nucleus_team_comparison(MONUSAC_TEAMS_PATH, nuclei, cl, idn)

In [ ]:
from monusac import show_matches_under_threshold

image_file = "TCGA-2Z-A9JG-01Z-00-DX1\TCGA-2Z-A9JG-01Z-00-DX1_4"
idt = 3

show_matches_under_threshold(image_file, idt, MONUSAC_TEAMS_PATH, MONUSAC_ANNOTATIONS_PATH)